## Deep kernel learning example on STEM data
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pycroscopy/DTMicroscope/blob/main/notebooks/STEM/2_active_learning_dkl_COLAB-Hackathon.ipynb)


In [ ]:
!pip install pyro5
!pip install scifireaders
!pip install sidpy
!pip install pynsid
!pip install git+https://github.com/pycroscopy/DTMicroscope.git@utk

In [ ]:
!run_server

# Active learning in STEM

One significant challenge in STEM is that it is not always possible to acquire spectroscopy across a dense grid of points due to the beam causing sample damage. Even in cases where there is little damage, this method is still highly wasteful. Instead, it is useful to be able to adaptively sample to maximize some property of interest. This example shows how deep kernel learning can be used for this adaptive sampling/optimization.

## Client side starts

In [ ]:
!pip install atomai
!pip install gpax

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import Pyro5.api
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from atomai.utils import get_coord_grid, extract_patches_and_spectra, extract_subimages
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gpax

In [ ]:
# Connect to the microscope server
uri = "PYRO:microscope.server@localhost:9091"
mic_server = Pyro5.api.Proxy(uri)


In [ ]:
### 2. Download data and register
!wget https://github.com/pycroscopy/DTMicroscope/raw/utk/data/STEM/SI/test_stem.h5
mic_server.initialize_microscope("STEM")
mic_server.register_data("test_stem.h5")


In [ ]:
mic = mic_server

In [ ]:
def load_image_and_features(img, window_size):
    coordinates = get_coord_grid(img, step = 1, return_dict=False)
    features_all, coords, _ = extract_subimages(img, coordinates, window_size)
    features_all = features_all[:,:,:,0]
    coords = np.array(coords, dtype=int)
    norm_ = lambda x: (x - x.min()) / x.ptp()
    features = norm_(features_all)
    return features, coords

In [ ]:
# Get overview image
array_list, shape, dtype = mic_server.get_overview_image()
img = np.array(array_list, dtype=dtype).reshape(shape)

n, d1, d2 = features.shape
X = features.reshape(n, d1*d2)
plt.imshow(img)# shape is 55,70
plt.scatter(60, 25, s=1, c='r')

In [ ]:
# use only 0.02% of grid data points as initial training points
(X_measured, X_unmeasured, indices_measured, indices_unmeasured) = train_test_split(
    X, indices_all, test_size=0.990, shuffle=True, random_state=3)

seed_points = len(X_measured)

print("Seed points: ", seed_points)

In [ ]:
energy_range_01 = [0.35, 0.55]# dipole mode
energy_range_02 = [0.6, 0.75]# edge mode
energy_range_03 = [0.8, 1.0]# bulk mode
spectral_img = mic.get_spectrum_image(channel_key="Channel_001")
E_axis = spectral_img.dim_2.values
e1a, e1b = abs(E_axis - energy_range_01[0]).argmin(), abs(E_axis - energy_range_01[1]).argmin()# 2 numbers
e2a, e2b = abs(E_axis - energy_range_02[0]).argmin(), abs(E_axis - energy_range_02[1]).argmin()# 2 numbers
e3a, e3b = abs(E_axis - energy_range_03[0]).argmin(), abs(E_axis - energy_range_03[1]).argmin()# 2 numbers


In [ ]:
reference_y_pred = np.sum(spectral_img[:, :, e1a:e1b], axis=2)
# Normalize the entire array
norm_ = lambda x: (x - np.min(x)) / np.ptp(x)
reference_y_pred = norm_(reference_y_pred)

In [ ]:
reference_y_pred.shape

In [ ]:
# veri
y_measured_unnor = []
for i in range (seed_points):
    # converted_coords = newexp.convert_coordinates(indices_measured[i,::-1], num_pix_x = 192, num_pix_y = 192)
    # wv, quick_fit, cx, chns = newexp.do_beps_specific(coordinates = converted_coords,
    #                                                 file_name = "BEPS_seed_{}".format(i))
    # amp_off, amp_on = sep_on_off(np.asarray(quick_fit)[0,0,])
    # pha_off, pha_on = sep_on_off(np.asarray(quick_fit)[0,3,])
    # current_y = loop_area(amp_off*np.cos(pha_off), 3)
    spectrum = mic.get_point_data("Channel_001", indices_measured[i, 0], indices_measured[i, 1])# should the index be reversed?
    # choose mean as the scalarizer function
    print(spectrum)
    current_y = spectrum[e1a:e1b].sum() 
    y_measured_unnor.append(current_y)
    # time.sleep(0.5)
    
norm_ = lambda x: (x - x.min()) / x.ptp()
y_measured = norm_(np.asarray(y_measured_unnor))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(img)
axes[0].scatter(indices_measured[:, 1], indices_measured[:, 0], c="r", marker = "X", s=30)
axes[0].set_title('Seed Points')

axes[1].imshow(img)
axes[1].scatter(indices_measured[:, 1], indices_measured[:, 0], c=y_measured, marker = "X", s=30)
axes[1].set_title('Seed Points')





In [ ]:
budget = 100

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error

def plot_acquisition_and_dkl_prediction(
    indices_unmeasured, obj, img, dkl, key1, X, indices_all, reference_scalarizer, file_name, step
):
    """
    Plots the acquisition function values, DKL predictions, the original image, 
    and the reference scalarizer, then saves the plot as a PNG file.

    Parameters:
    - indices_unmeasured: np.ndarray of unmeasured indices.
    - obj: np.ndarray of acquisition function values.
    - img: np.ndarray, base image to overlay predictions on.
    - dkl: DKL model instance with a predict method.
    - key1: Random state or key used in the DKL predict function.
    - X: np.ndarray of input data points for DKL prediction.
    - indices_all: np.ndarray of all indices used for prediction.
    - reference_scalarizer: np.ndarray, reference scalarizer values.
    - file_name: str, the file name (without extension) to save the plot as PNG.
    """
    fig, ax = plt.subplots(2, 2, figsize=(12, 10))  # Create a 2x2 grid of subplots

    # Create an image to display acquisition function values using imshow
    acq_img = np.zeros_like(img, dtype=float)  # Initialize a grid of zeros with the same shape as the original image

    # Fill in the acquisition values at the unmeasured indices
    for idx, (i, j) in enumerate(indices_unmeasured):
        acq_img[i, j] = obj[idx]

    # Plot the acquisition function values using imshow
    im1 = ax[0, 0].imshow(acq_img, cmap='viridis')
    cbar1 = fig.colorbar(im1, ax=ax[0, 0], orientation='vertical', fraction=0.046, pad=0.04)
    cbar1.set_label('Acquisition Function Value')

    # Highlight the next point to measure
    next_point = indices_unmeasured[obj.argmax()]
    ax[0, 0].scatter(next_point[1], next_point[0], marker='x', c='k', s=100)
    ax[0, 0].set_title("Acquisition Function Values")
    ax[0, 0].set_xlabel('X Index')
    ax[0, 0].set_ylabel('Y Index')

    # Plot the DKL prediction
    y_pred_img = np.zeros((img.shape[0], img.shape[1]))
    y_pred_mean, y_pred_std = dkl.predict(key1, np.array(X))

    # Fill the prediction image with predicted mean values
    for j in range(len(indices_all)):
        y_pred_img[indices_all[j][0], indices_all[j][1]] = y_pred_mean[j]

    im2 = ax[0, 1].imshow(y_pred_img, cmap='plasma')
    cbar2 = fig.colorbar(im2, ax=ax[0, 1], orientation='vertical', fraction=0.046, pad=0.04)
    cbar2.set_label('Predicted Mean Value')

    ax[0, 1].set_title("DKL Prediction")
    ax[0, 1].set_xlabel('X Index')
    ax[0, 1].set_ylabel('Y Index')

    # Plot the original image
    ax[1, 0].imshow(img, cmap='gray')
    ax[1, 0].set_title("Original Image")
    ax[1, 0].set_xlabel('X Index')
    ax[1, 0].set_ylabel('Y Index')

    # Plot the reference scalarizer
    im3 = ax[1, 1].imshow(reference_scalarizer, cmap='inferno')
    cbar3 = fig.colorbar(im3, ax=ax[1, 1], orientation='vertical', fraction=0.046, pad=0.04)
    cbar3.set_label('Reference Scalarizer')

    ax[1, 1].set_title("Reference Scalarizer")
    ax[1, 1].set_xlabel('X Index')
    ax[1, 1].set_ylabel('Y Index')

    # Calculate MSE between the DKL prediction and the reference scalarizer
    mse = mean_squared_error(reference_scalarizer.flatten(), y_pred_img.flatten())
    print(f"Mean Squared Error between DKL Prediction and Reference Scalarizer: {mse:.4f}")
    fig.suptitle(f" Step_{step}_Mean Squared Error: {mse:.4f}", fontsize=16)
    plt.tight_layout()
    plt.savefig(f"{file_name} + 'mse' {mse:.4f} .png")
    plt.close(fig)  # Close the figure to free memory

data_dim = X_measured.shape[-1]
key1, key2 = gpax.utils.get_keys()

In [ ]:
for i in range(budget):
            dkl = gpax.viDKL(data_dim, 2)
            # import pdb; pdb.set_trace()
            dkl.fit(key1, np.array(X_measured), np.array(y_measured), num_steps=100, step_size=0.05)
            obj = gpax.acquisition.UCB(key2, dkl, np.array(X_unmeasured), beta=0.25, maximize=True)
            next_point_idx = obj.argmax()
            measured_point = mic.get_point_data("Channel_001", indices_measured[i, 0], indices_measured[i, 1])
            measured_point = measured_point[e1a:e1b].sum() # scalarizer
            
            plot_acquisition_and_dkl_prediction(indices_unmeasured, obj, img, dkl, key1, X, indices_all, reference_y_pred, f"{out_dir}/dkl_pred_acq_{i}", i)

            
            
            X_measured = np.append(np.array(X_measured), np.array(X_unmeasured)[next_point_idx][None], 0)
            X_unmeasured = np.delete(np.array(X_unmeasured), next_point_idx, 0)
            y_measured_unnor = np.append(np.array(y_measured_unnor), measured_point)
            y_measured = norm_(np.asarray(y_measured_unnor))
            # y_unmeasured = np.delete(np.array(y_unmeasured), next_point_idx)
            indices_measured = np.append(np.array(indices_measured), np.array(indices_unmeasured)[next_point_idx][None], 0)
            indices_unmeasured = np.delete(np.array(indices_unmeasured), next_point_idx, 0)
            

                    